# Unix Timestamp

### Problemas comunes con los datos de tipo timestamp o datetime

Las fechas se guardan como valores numericos, pero hay que definir un monton de cosas arbitrarias:

  1. No esta clara la zona horaria (pueden ser naive o aware). referencia geografica. Mostra [mapa](https://es.wikipedia.org/wiki/Tiempo_universal_coordinado#/media/Archivo:World_Time_Zones_Map.png)
  2. No esta claro desde cuando se cuentan. Referencia temporal cero.
  3. No esta claro como se cuentan. Que dia es el primero de la semana? Lunes, Domingo? Se empieza en 1 o en 0? Que es la primer semana de un año?
  4. No esta claro las convenciones de lenguaje. Que fecha es 06/05/12? Es 6 de mayo de 1912? es 5 de junio de 2012?
  5. ¿Como hago para adaptar los print al idioma o convencion local?

De todo esto se encarga la libreria datetime. Esta libreria esta incluida en numpy y en pandas, pero en pandas tiene algunas diferencias tecnicas para que pueda servir de indice, por eso se llama timestamp. 

### Como se solucionan los problemas

1. Las fechas por default no tienen zona horaria, son naive, pero se les puede agregar con la informacion timezone que permite representar la informacion de la zona horaria como una propiedad de la fecha. En este caso se dice que es una fecha conciente (aware). Sino asume que esta en el uso horario local.
2. Las fecha se guardan como el numero de segundos que transcurrieron desde el 1ero de Enero de 1970 (en grenwich). Son el numero de segundos desde ese momento. El resto es adaptaciones de formato que hace la libreria.
3. Hay convenciones. En general se toma la fecha como segundos, horas, dias, meses y años. Pero hay metodos que permiten cuantificar de otras maneras, por ejemplo en semanas. Hay mas de un formato y depende que metodo usemos como representa la informacion, pero el numero es siempre el mismo.
4. Lo mismo que el punto anterior. Segun que metodos usemos podemos usar una convencion u otra. Hay metodos que permiten configurar formatos a medida y hay metodos que recuperan las convenciones de la configuracion local sel sistema operativo. 
5. Lo mismo.

### Principales tipos de fechas (tienen diferentes metodos)

- period / timedelta : Representa un tiempo transcurrido, es una diferencia entre dos fechas. Guarda la cantidad de dias, segundos y microsegundos, y tiene todos los metodos usuales para representar esa informacion. Es util cuando se quiere hacer operaciones entre tiempos. Como sumar, restar, comparar, o hacer intervalos periodicos. 
- time : Esta pensado para representar tiempos menores a un dia, tiene los metodos asociados.
- date : Esta pensado para representar las fechas en funcion de los dias desde 01/01/1970. Tiene los metodos asociados.
- timedate : combina toda la informacion de una fecha, es el equivalente a timestamp de pandas.
- timezone : sirve para manipular la informacion de la zona horaria.

  - Convenciones de formato de fechas: https://strftime.org/
  - Documentacion completa en español: https://docs.python.org/es/3/library/datetime.html


In [4]:
from datetime import datetime

t = datetime.now()
print (t.timestamp())

1665709675.547432


In [9]:
print (datetime.now().isoformat())
print (datetime.now().ctime())

2022-10-13T22:12:49.226620
Thu Oct 13 22:12:49 2022


In [12]:
import pytz

for tz in pytz.all_timezones:
    print(tz)

Africa/Abidjan
Africa/Accra
Africa/Addis_Ababa
Africa/Algiers
Africa/Asmara
Africa/Asmera
Africa/Bamako
Africa/Bangui
Africa/Banjul
Africa/Bissau
Africa/Blantyre
Africa/Brazzaville
Africa/Bujumbura
Africa/Cairo
Africa/Casablanca
Africa/Ceuta
Africa/Conakry
Africa/Dakar
Africa/Dar_es_Salaam
Africa/Djibouti
Africa/Douala
Africa/El_Aaiun
Africa/Freetown
Africa/Gaborone
Africa/Harare
Africa/Johannesburg
Africa/Juba
Africa/Kampala
Africa/Khartoum
Africa/Kigali
Africa/Kinshasa
Africa/Lagos
Africa/Libreville
Africa/Lome
Africa/Luanda
Africa/Lubumbashi
Africa/Lusaka
Africa/Malabo
Africa/Maputo
Africa/Maseru
Africa/Mbabane
Africa/Mogadishu
Africa/Monrovia
Africa/Nairobi
Africa/Ndjamena
Africa/Niamey
Africa/Nouakchott
Africa/Ouagadougou
Africa/Porto-Novo
Africa/Sao_Tome
Africa/Timbuktu
Africa/Tripoli
Africa/Tunis
Africa/Windhoek
America/Adak
America/Anchorage
America/Anguilla
America/Antigua
America/Araguaina
America/Argentina/Buenos_Aires
America/Argentina/Catamarca
America/Argentina/ComodRivad

In [16]:
t_a = datetime.now(pytz.timezone('Australia/Perth'))
t_a
t_a.timestamp()
print (t_a)

2022-10-14 09:19:29.381520+08:00


In [22]:
t = datetime.now()
t.month_name()

AttributeError: 'datetime.datetime' object has no attribute 'month_name'

In [33]:
import pandas as pd
t = pd.Timestamp.now()
t.month_name()

t.strftime("%W")

'41'

In [44]:
fechas_str = ["13/03/11","11/2/2001","1-4-2001","110101","01Jan2001"]
data = pd.DataFrame({"Fechas Str":fechas_str})

data['Fecha'] = pd.to_datetime(data['Fechas Str'])
data


,Fechas Str,Fecha
0,13/03/11,2011-03-13
1,11/2/2001,2001-11-02
2,1-4-2001,2001-01-04
3,110101,2001-11-01
4,01Jan2001,2001-01-01


In [40]:
print (pd.to_datetime('170901 100500', format='%d%m%y %H%M%S'))
print (pd.to_datetime('20170109 100500-10:30', format='%Y%d%m %H%M%S%z'))

2001-09-17 10:05:00
2017-09-01 10:05:00-10:30


## Situacion a resolver

Tenemos un dataset con notas de alumnos (los numeros son todos inventados pero son decimales y van de 0 a 10). Tenemos que pasar esas notas a categorias para los boletines donde:

0-2 es mal
2-4 es regular
4-7 es bien
7-9 es muy bien
9-10 es excelente.

y ademas despues sabemos que bien, muy bien y excelente son aprobados, regular y mal no. 

Como modificamos el dataset para hacerlo de manera rapida. 

Lo que necesitamos es pasar de una variable cuantitativa a una cualitativa y despues transformar y agrupar las categorias.


In [46]:
import numpy as np
import pandas as pd

In [47]:
# Generamos un dataset ficticio.

Mal = 2
Regular = 4
Bien = 7
Muy_Bien = 9
Excelente = 10

notas = np.random.randint(0,1000,80)/100  # Generamos enteros entre 0 y 10 ochenta veces
data = pd.DataFrame({"Notas":notas})
data.head(10)

,Notas
0,3.11
1,6.47
2,9.07
3,1.58
4,9.97
5,1.14
6,5.86
7,8.33
8,8.01
9,5.56


In [48]:
# Vamos a tratar de resolver la primer parte.
# Para eso queremos convertir una variable numerica en una categorica con rangos preestablecidos. La funcion pd.cut hace eso, pero necesita saber los bins
bins = [0, Mal, Regular, Bien, Muy_Bien, Excelente+0.01] # Recordar que ya definimos que es Mal, Bien, etc.
# Vamos a generar etiquetas (strings) que se correspondan a esas categorias
group_labels = ["Mal","Regular","Bien","Muy Bien","Excelente"]

Notas_categorizadas = pd.cut(data["Notas"], bins, right=True, include_lowest=True, labels=group_labels) # , right=False , labels=group_labels
Notas_categorizadas


0       Regular
1          Bien
2     Excelente
3           Mal
4     Excelente
        ...    
75    Excelente
76      Regular
77         Bien
78      Regular
79      Regular
Name: Notas, Length: 80, dtype: category
Categories (5, object): ['Mal' < 'Regular' < 'Bien' < 'Muy Bien' < 'Excelente']

In [50]:
Notas_categorizadas.value_counts()
data["Categorias"] = Notas_categorizadas

In [51]:
dict_aprobacion = {"Mal":"Desaprobado","Regular":"Desaprobado","Bien":"Aprobado","Muy Bien":"Aprobado","Excelente":"Aprobado"}
data["Condicion"] = data["Categorias"].map(dict_aprobacion)
data.head(10)

,Notas,Categorias,Condicion
0,3.11,Regular,Desaprobado
1,6.47,Bien,Aprobado
2,9.07,Excelente,Aprobado
3,1.58,Mal,Desaprobado
4,9.97,Excelente,Aprobado
5,1.14,Mal,Desaprobado
6,5.86,Bien,Aprobado
7,8.33,Muy Bien,Aprobado
8,8.01,Muy Bien,Aprobado
9,5.56,Bien,Aprobado


In [52]:
data_dummy = pd.get_dummies(data, drop_first=True)

In [53]:
data_dummy

,Notas,Categorias_Regular,Categorias_Bien,Categorias_Muy Bien,Categorias_Excelente,Condicion_Desaprobado
0,3.11,1,0,0,0,1
1,6.47,0,1,0,0,0
2,9.07,0,0,0,1,0
3,1.58,0,0,0,0,1
4,9.97,0,0,0,1,0
...,...,...,...,...,...,...
75,9.02,0,0,0,1,0
76,2.75,1,0,0,0,1
77,4.02,0,1,0,0,0
78,2.97,1,0,0,0,1
